In [1]:
import pandas as pd
from os.path  import join
import os
from shutil import rmtree, copyfile

In [2]:
RANDOM = 20241203
IMAGE_H = 1486
IMAGE_W = 2048

### Class

In [3]:
trash_dict = {
    1: "PET (transparent) (green)",
    2: "PET (transparent) (brown)",
    3: "PET (transparent) (blue)",
    4: "PET (transparent)",
    5: "PET (transparent) (dark blue)",
    6: "PET (black)",
    7: "PET (white)",
    8: "PET (sticker)",
    9: "PET (flacon)",
    10: "PET (household chemicals)",
    11: "PND (household chemicals)",
    12: "PND packet",
    13: "Other plastic",
    14: "Other plastic (transparent)",
    15: "Not plastic"
}

with open ('trash_dict', 'w') as file:
    file.write(str(trash_dict))

trash_dict

{1: 'PET (transparent) (green)',
 2: 'PET (transparent) (brown)',
 3: 'PET (transparent) (blue)',
 4: 'PET (transparent)',
 5: 'PET (transparent) (dark blue)',
 6: 'PET (black)',
 7: 'PET (white)',
 8: 'PET (sticker)',
 9: 'PET (flacon)',
 10: 'PET (household chemicals)',
 11: 'PND (household chemicals)',
 12: 'PND packet',
 13: 'Other plastic',
 14: 'Other plastic (transparent)',
 15: 'Not plastic'}

In [4]:
trash_dict_yolo = dict(zip([x -1 for x in trash_dict.keys()], trash_dict.values()))

with open ('trash_dict_yolo', 'w') as file:
    file.write(str(trash_dict_yolo))

trash_dict_yolo

{0: 'PET (transparent) (green)',
 1: 'PET (transparent) (brown)',
 2: 'PET (transparent) (blue)',
 3: 'PET (transparent)',
 4: 'PET (transparent) (dark blue)',
 5: 'PET (black)',
 6: 'PET (white)',
 7: 'PET (sticker)',
 8: 'PET (flacon)',
 9: 'PET (household chemicals)',
 10: 'PND (household chemicals)',
 11: 'PND packet',
 12: 'Other plastic',
 13: 'Other plastic (transparent)',
 14: 'Not plastic'}

### Train

In [5]:
train_df = pd.read_csv('/kaggle/input/waste-detection/train.csv')
train_df

,file_name,bbox,category_id
0,000001.jpg,"[840.0, 0.0, 176.0, 124.0]",4.0
1,000001.jpg,"[612.0, 306.0, 383.0, 397.0]",2.0
2,000001.jpg,"[990.92, 551.0, 105.00000000000011, 186.0]",4.0
3,000002.jpg,"[1000.0, 614.0, 98.0, 178.0]",4.0
4,000002.jpg,"[605.0, 358.0, 402.0, 409.0]",2.0
...,...,...,...
32260,008999.jpg,"[1220.0, 0.0, 167.0, 236.0]",2.0
32261,009000.jpg,"[1218.0, 40.0, 169.0, 254.0]",2.0
32262,009000.jpg,"[320.0, 631.0, 527.0, 218.0]",1.0
32263,009000.jpg,"[1157.0, 1454.0, 65.0, 31.0]",2.0


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32265 entries, 0 to 32264
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    32265 non-null  object 
 1   bbox         32148 non-null  object 
 2   category_id  32148 non-null  float64
dtypes: float64(1), object(2)
memory usage: 756.3+ KB


In [7]:
train_df.isnull().sum()

file_name        0
bbox           117
category_id    117
dtype: int64

In [8]:
train_df=train_df.dropna()

In [9]:
train_df.describe()

,category_id
count,32148.000000
mean,6.369728
std,4.881510
min,1.000000
25%,2.000000
50%,4.000000
75%,11.000000
max,15.000000


In [10]:
def bbox_to_yolo(x1, y1, w, h): ## coco_to_yolo
    return [((2*x1 + w)/(2*IMAGE_W)) , ((2*y1 + h)/(2*IMAGE_H)), w/IMAGE_W, h/IMAGE_H]

In [11]:
train_df.loc[:, 'bbox'] = train_df.bbox.apply(eval)
print(train_df['bbox'])
train_df['bbox'].apply(type).value_counts()

0                        [840.0, 0.0, 176.0, 124.0]
1                      [612.0, 306.0, 383.0, 397.0]
2        [990.92, 551.0, 105.00000000000011, 186.0]
3                      [1000.0, 614.0, 98.0, 178.0]
4                      [605.0, 358.0, 402.0, 409.0]
                            ...                    
32260                   [1220.0, 0.0, 167.0, 236.0]
32261                  [1218.0, 40.0, 169.0, 254.0]
32262                  [320.0, 631.0, 527.0, 218.0]
32263                  [1157.0, 1454.0, 65.0, 31.0]
32264                 [583.0, 1118.0, 281.0, 306.0]
Name: bbox, Length: 32148, dtype: object


bbox
<class 'list'>    32148
Name: count, dtype: int64

In [12]:
train_df = train_df.astype({'category_id': 'Int32'})
train_df['category_id']

0        4
1        2
2        4
3        4
4        2
        ..
32260    2
32261    2
32262    1
32263    2
32264    1
Name: category_id, Length: 32148, dtype: Int32

In [13]:
train_df.loc[:, 'yolo'] = train_df.bbox.map(lambda x: bbox_to_yolo(*x))
train_df['yaml'] = train_df.category_id.map(lambda x: str(x-1)) + ' ' + train_df.yolo.map(lambda y: ' '.join([str(x) for x in y]))

train_df

,file_name,bbox,category_id,yolo,yaml
0,000001.jpg,"[840.0, 0.0, 176.0, 124.0]",4,"[0.453125, 0.04172274562584118, 0.0859375, 0.0...",3 0.453125 0.04172274562584118 0.0859375 0.083...
1,000001.jpg,"[612.0, 306.0, 383.0, 397.0]",2,"[0.392333984375, 0.3395020188425303, 0.1870117...",1 0.392333984375 0.3395020188425303 0.18701171...
2,000001.jpg,"[990.92, 551.0, 105.00000000000011, 186.0]",4,"[0.509482421875, 0.43337819650067294, 0.051269...",3 0.509482421875 0.43337819650067294 0.0512695...
3,000002.jpg,"[1000.0, 614.0, 98.0, 178.0]",4,"[0.51220703125, 0.4730820995962315, 0.04785156...",3 0.51220703125 0.4730820995962315 0.047851562...
4,000002.jpg,"[605.0, 358.0, 402.0, 409.0]",2,"[0.3935546875, 0.3785329744279946, 0.196289062...",1 0.3935546875 0.3785329744279946 0.1962890625...
...,...,...,...,...,...
32260,008999.jpg,"[1220.0, 0.0, 167.0, 236.0]",2,"[0.636474609375, 0.07940780619111709, 0.081542...",1 0.636474609375 0.07940780619111709 0.0815429...
32261,009000.jpg,"[1218.0, 40.0, 169.0, 254.0]",2,"[0.635986328125, 0.11238223418573351, 0.082519...",1 0.635986328125 0.11238223418573351 0.0825195...
32262,009000.jpg,"[320.0, 631.0, 527.0, 218.0]",1,"[0.284912109375, 0.4979811574697174, 0.2573242...",0 0.284912109375 0.4979811574697174 0.25732421...
32263,009000.jpg,"[1157.0, 1454.0, 65.0, 31.0]",2,"[0.580810546875, 0.9888963660834454, 0.0317382...",1 0.580810546875 0.9888963660834454 0.03173828...


In [14]:
train_df.to_csv('train_df.csv')

## Images

In [15]:
images_complete = sorted(os.listdir('/kaggle/input/waste-detection/images/'))
##images_complete = images_complete[:10] #demo
images_complete[:10] ##

['000001.jpg',
 '000002.jpg',
 '000003.jpg',
 '000004.jpg',
 '000005.jpg',
 '000006.jpg',
 '000007.jpg',
 '000008.jpg',
 '000009.jpg',
 '000010.jpg']

In [ ]:
images_input = '/kaggle/input/waste-detection/images/'

images_output = '/kaggle/working/images/'
os.makedirs(images_output, exist_ok=True) 

for image in images_complete:
    copyfile(join(images_input, image), join(images_output, image))

## Labels

In [ ]:
labels_path = '/kaggle/working/labels/'
os.makedirs(labels_path, exist_ok=True) 

for image in images_complete:
    with open(join(labels_path, os.path.splitext(image)[0]+'.txt'), 'w') as file:
        file.write('\n'.join(list(train_df.query('file_name == @image').yaml)))